In [1]:
!ls

datalab


In [4]:
!ls

datalab  drive


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time

tf.reset_default_graph()
start_time = time.time()

def initial_parameters():
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
    w1 = tf.get_variable('w1', [4, 4, 3, 8], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
    w2 = tf.get_variable('w2', [2, 2, 8, 16], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
    w3 = tf.get_variable('w3', [256, 1], dtype=tf.float32, initializer=tf.zeros_initializer)
    parameters = {'w1': w1, 'w2': w2, 'w3': w3}
    return parameters


def forward_propagation(X, parameters):
    w1 = parameters['w1']
    w2 = parameters['w2']
    w3 = parameters['w3']
    Z1 = tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1, ksize=[1, 8, 8, 1], strides=[1, 8, 8, 1], padding='SAME')
    Z2 = tf.nn.conv2d(P1, w2, strides=[1, 1, 1, 1], padding="SAME")
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.matmul(P2, w3)
#     dropout = tf.layers.dropout(inputs=Z3, rate=0.4, training=True)
#     Z3 = tf.layers.dense(inputs=dropout, units=1, name='Z3')

    # Z3 = tf.layers.Dense(inputs=P2, units=1, name='w3', activation=None)
    return Z3


def compute_cost(Z3, Y):
    '''
    利用tf.nn 计算损失函数
    与nn不同在与 数据的排列方式不同了，nn是样本再axis1，这个是在axis0
    '''
    return tf.reduce_mean(tf.square(Z3-Y), name='compute_cost')


def creat_place_holder(nH0, nW0, n_C0, n_y):
    '''
    创建x,y placeholder
    还是数据排列方式不同了
    '''
    X = tf.placeholder(dtype=tf.float32, shape=[None, nH0, nW0, n_C0], name='X')
    Y = tf.placeholder(dtype=tf.float32, shape=[None, n_y], name='Y')
    return X, Y


def load_data():
    '''
    载入数据并处理  包括归一化 和 转变独热码
    数据不同reshape了
    '''
    train_x = np.array(np.load('drive/deep_learning/datasets/train_x.zip_files/train_x.npy'))
    train_y = np.array(np.load('drive/deep_learning/datasets/train_y.npy'))
    print(train_y.shape)
    return train_x, train_y


def mini_batch(X, Y, mini_batch_size, seed=0):
    '''
    将数据划分为minib
    也根据数据发生改变了
    '''
    m = X.shape[0]
    permutation = np.random.permutation(m)
    shuffled_X = X[permutation, :, :, :]
    shuffled_Y = Y[permutation, :]
    batch_num = m // mini_batch_size
    batches = []
    for i in range(batch_num):
        X_batch = shuffled_X[i*mini_batch_size:i*mini_batch_size+mini_batch_size, :, :, :]
        Y_batch = shuffled_Y[i*mini_batch_size:i*mini_batch_size+mini_batch_size, :]
        batch = (X_batch, Y_batch)
        batches.append(batch)
    if m % mini_batch_size != 0:
        X_batch = shuffled_X[batch_num*mini_batch_size:m, :, :, :]
        Y_batch = shuffled_Y[batch_num*mini_batch_size:m, :]
        batch = (X_batch, Y_batch)
        batches.append(batch)
    return batches


if __name__ == '__main__':
    #最后结果训练正确率97%，测试85
    #测试计算图也发生了一点变化，axis defalt0变为1
    train_set_x, train_set_y = load_data()
    y_mean = np.mean(train_set_y)
    y_std = np.mean(train_set_y)
    tf.constant(y_mean, name='y_mean')
    tf.constant(y_std, name='y_std')
    train_set_y = (train_set_y-y_mean) / y_std
    m, n_H0, n_W0, n_C0 = train_set_x.shape
    n_y = train_set_y.shape[1]
    costs=[] #画图用 存放损失函数
    batches = mini_batch(train_set_x[:601,:,:,:], train_set_y[:601,:], 16)

    X, Y = creat_place_holder(n_H0, n_W0, n_C0, n_y)

    parameter = initial_parameters()

    Z3 = forward_propagation(X, parameter)

    cost = compute_cost(Z3, Y)

    optimer = tf.train.AdamOptimizer(learning_rate=0.00005).minimize(cost, name='op_cost')
    print(optimer)
    init = tf.global_variables_initializer()
    num_minibatches = int(train_set_y.shape[0]/16)
    with tf.Session() as sess:
        sess.run(init)
        for i in range(5000):
            epoch_loss = 0
            for mini_X, mini_Y in batches:
                _, loss = sess.run([optimer, cost], feed_dict={X: mini_X, Y: mini_Y})
                epoch_loss += loss / num_minibatches
            if i % 20 == 0:
                print('epoch', i, ":", epoch_loss)
                costs.append(epoch_loss)
        parameter = sess.run(parameter)
#         print(parameter['w1'][1, 1, 1])
        tf.add_to_collection('forward', Z3)
        saver = tf.train.Saver()
        saver.save(sess, 'drive/deep_learning/save_model/my_model')
        Z = sess.run(Z3, feed_dict={X:train_set_x[601:,:,:,:]})
        result_z = Z*y_std+y_mean
        big_z = train_set_y*y_std + y_mean
        print(abs(result_z-big_z[601:,]))

  
    end_time = time.time()
    print(end_time - start_time)
    plt.plot(costs)
    plt.show()

In [15]:
import tensorflow as tf
tf.test.gpu_device_name() 


'/device:GPU:0'

In [8]:


sess = tf.Session()
## 这里是恢复graph
saver = tf.train.import_meta_graph('drive/deep_learning/save_model/my_model.meta')
## 这里是恢复各个权重参数
saver.restore(sess, tf.train.latest_checkpoint('drive/deep_learning/save_model'))

## 获取输入的tensor
graph = tf.get_default_graph()

X = graph.get_tensor_by_name('X:0')

## 给定数据，运行encoder_op
a = sess.run(Z3, feed_dict={X:train_set_x[:60,:,:,:]})
y_mean, y_std = sess.run(['y_mean:0', 'y_std:0'])
result_z = a*y_std+y_mean
big_z = train_set_y*y_std + y_mean
print(result_z-big_z[:60,:])

INFO:tensorflow:Restoring parameters from drive/deep_learning/save_model/my_model
[[  17.07714844]
 [  25.26428223]
 [ 210.87390137]
 [ -96.91748047]
 [   7.70330811]
 [  55.87390137]
 [-323.12609863]
 [-167.12609863]
 [ 235.87390137]
 [ -16.73623657]
 [-148.12609863]
 [  37.13110352]
 [  61.87390137]
 [ -42.55581665]
 [  25.00445557]
 [ 195.87390137]
 [   3.92303467]
 [  -9.74719238]
 [ -18.12609863]
 [ -31.12609863]
 [ 202.87390137]
 [ -97.26034546]
 [ 211.87390137]
 [ 154.87390137]
 [ 172.87390137]
 [ -18.68243408]
 [ 210.87390137]
 [  27.38006592]
 [ -37.40106201]
 [ -71.35406494]
 [ -57.61669922]
 [  29.77026367]
 [ -76.3215332 ]
 [  53.87390137]
 [  12.59274292]
 [  17.12310791]
 [ -59.47045898]
 [ 169.87390137]
 [ 193.87390137]
 [-231.12609863]
 [ -22.77990723]
 [  31.36752319]
 [-104.12609863]
 [ -21.01812744]
 [ 206.87390137]
 [ 287.87390137]
 [ 267.87390137]
 [ -44.88098145]
 [  30.42721558]
 [ -86.39294434]
 [ -65.61010742]
 [-217.12609863]
 [-128.9942627 ]
 [ -53.12609863]


In [36]:
import tensorflow as tf
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('drive/deep_learning/save_model/my_model.meta')
    saver.restore(sess, tf.train.latest_checkpoint('drive/deep_learning/save_model'))
    graph = tf.get_default_graph()
    cost = graph.get_tensor_by_name('compute_cost:0')
    print(cost)
    optimer = graph.get_operation_by_name('op_cost')
    #optimer2 = tf.train.AdadeltaOptimizer(learning_rate=0.00001).minimize(cost)
    print(optimer)
    for i in range(200):
        epoch_loss = 0
        for mini_X, mini_Y in batches:
            _, loss = sess.run([optimer, cost], feed_dict={X: mini_X, Y: mini_Y})
            epoch_loss += loss / num_minibatches
        if i % 20 == 0:
            print('epoch', i, ":", epoch_loss)
            costs.append(epoch_loss)
            
    #已经有saver就不能创建了        
    saver.save(sess, './drive/deep_learning/save_model/my_model')
#     saver = tf.train.Saver()
#     saver.save(sess, './drive/deep_learning/save_model/my_model')
    Z = sess.run(Z3, feed_dict={X:train_set_x[641:,:,:,:]})
    y_mean = sess.run('y_mean:0')
    y_std = sess.run('y_std:0')
    result_z = Z*y_std+y_mean
    big_z = train_set_y*y_std + y_mean
    print(abs(result_z-big_z[641:,]))

INFO:tensorflow:Restoring parameters from drive/deep_learning/save_model/my_model
Tensor("compute_cost:0", shape=(), dtype=float32)
name: "op_cost"
op: "NoOp"
input: "^op_cost/update_w1/ApplyAdam"
input: "^op_cost/update_w2/ApplyAdam"
input: "^op_cost/update_w3/ApplyAdam"
input: "^op_cost/update_Z3/kernel/ApplyAdam"
input: "^op_cost/update_Z3/bias/ApplyAdam"
input: "^op_cost/Assign"
input: "^op_cost/Assign_1"

epoch 0 : 0.04265898015970985
epoch 20 : 0.04586300955131828
epoch 40 : 0.038052263736192665
epoch 60 : 0.04499051999300717
epoch 80 : 0.043496449511232126


KeyboardInterrupt: ignored

In [0]:
a = tf.Variable(3)

In [28]:
a

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>

In [35]:
with tf.Session() as sess:
  sess.run(a.assign(30))
  print(sess.run(a))
  print(sess.run(a.initializer))
  print(sess.run(a))
  #说明了一执行初始化，变量值就变成最初的值了

30
None
3
